# Electrolyte Optimiser (PumpBot V2)

Update experiment details in [yaml file](config/hardware_config.yaml). Then run the below code block **once per session**.

In [1]:
%load_ext autoreload
%autoreload 2

from src.scheduler import scheduler
config_path = "config/hardware_config.yaml"

device = scheduler(config_path)

INFO:root:Configuring pump controller serial port..
INFO:root:Attempting to open pump controller serial port..
INFO:root:Simulated connection to pump controller established.
INFO:root:Simulated connection to pump controller established.
INFO:root:Temperature regulator PID mode successfully configured.
INFO:root:Temperature regulator dead band settings successfully configured.
INFO:root:Temperature regulator voltage alarm settings successfully configured.
INFO:root:Temperature regulator current alarm settings successfully configured.
INFO:root:Temperature Sensor #1 successfully configured.
INFO:root:Temperature sensor #2 successfully configured.
INFO:root:Successfully updated steinhart coefficients for temperature sensor #1.
INFO:root:Successfully updated steinhart coefficients for temperature sensor #2.
INFO:root:Temperature regulator fan settings successfully configured.
INFO:root:Resetting AC and DC dataframes..


## Run the next code block to a single perform experiment.

In [2]:
device.run_basic_experiment(deprime=True)

INFO:root:Temperature regulator set to cooling mode.
INFO:root:Peltier target temperature set to 0.0C.
INFO:src.scheduler:Priming lines first..
INFO:src.scheduler:[DEBUG]: A, 1
INFO:src.scheduler:Dosing Chem1: 2.000 ml on A[1]
INFO:src.scheduler:Primed Chem1 with 2.0ml.
INFO:src.scheduler:Transferring 2.4 to cell..
INFO:src.scheduler:Transferring 2.4 to waste #1..
INFO:src.scheduler:[DEBUG]: A, 2
INFO:src.scheduler:Dosing Chem2: 2.000 ml on A[2]
INFO:src.scheduler:Primed Chem2 with 2.0ml.
INFO:src.scheduler:Transferring 2.4 to cell..
INFO:src.scheduler:Transferring 2.4 to waste #1..
INFO:src.scheduler:[DEBUG]: A, 3
INFO:src.scheduler:Dosing Chem3: 2.000 ml on A[3]
INFO:src.scheduler:Primed Chem3 with 2.0ml.
INFO:src.scheduler:Transferring 2.4 to cell..
INFO:src.scheduler:Transferring 2.4 to waste #1..
INFO:src.scheduler:[DEBUG]: A, 4
INFO:src.scheduler:Dosing Chem4: 2.000 ml on A[4]
INFO:src.scheduler:Primed Chem4 with 2.0ml.
INFO:src.scheduler:Transferring 2.4 to cell..
INFO:src.sched